# Notebook de referência 

Nome: Arthur Baia


## Instruções:


Treinar e medir a acurácia de um modelo BERT (ou variantes) para classificação binária usando o dataset do IMDB (20k/5k amostras de treino/validação).

Importante: 
- Deve-se implementar o próprio laço de treinamento.
- Implementar o acumulo de gradiente.

Dicas:
- BERT geralmente costuma aprender bem uma tarefa com poucas épocas (de 3 a 5 épocas). Se tiver demorando mais de 5 épocas para chegar em 80% de acurácia, ajuste os hiperparametros.

- Solução para erro de memória:
  - Usar bfloat16 permite quase dobrar o batch size

Opcional:
- Pode-se usar a função trainer da biblioteca Transformers/HuggingFace para verificar se seu laço de treinamento está correto. Note que ainda assim é obrigatório implementar o laço próprio.

- Usar pytorch lightning. Para entender como o pytorch lightning funciona, veja uma implementação simplificada no notebook `06_01_Treino_Validação_MNIST_Lightning_Lite.ipynb`

# Fixando a seed

In [1]:
import random
import torch
import torch.nn.functional as F
import numpy as np

In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [3]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)

## Preparando Dados

Primeiro, fazemos download do dataset:

In [4]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (20k exemplos) e validação (5k exemplos) artificialmente.

In [5]:
import os

max_valid = 5000

def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

x_valid = x_train[-max_valid:]
y_valid = y_train[-max_valid:]
x_train = x_train[:-max_valid]
y_train = y_train[:-max_valid]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('3 primeiras amostras validação:')
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print('3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
False So, this movie has been hailed, glorified, and carried to incredible heights. But in the end what is
False "Ghost of Dragstrip Hollow" appears to take place in a spotless netherworld, an era long gone by, wh
True Just after I saw the movie, the true magic feeling of the Walt Disney movies came up in me and I rea
3 últimas amostras treino:
False First off - there's absolutely no flirting going on in this film - with Anthony or anyone else. Thes
True I enjoyed Still Crazy more than any film I have seen in years. A successful band from the 70's decid
True Jerome Crabbe has the lead role in this movie. I saw this movie 6 times and I still am not tired of 
3 primeiras amostras validação:
True The memory of the "The Last Hunt" has stuck with me since I saw it in 1956 when I was 13. It is a mo
True Say what you want about Andy Milligan - but if his family was even 10% as der

In [6]:
!pip install transformers

In [7]:
import transformers
from transformers import (
    BertTokenizer,
    BertForSequenceClassification
)
import tqdm

In [8]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [9]:
train_encodings = tokenizer(x_train, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(x_valid, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(x_test, truncation=True, padding=True, max_length=512)

In [10]:
class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # transform label to  long tensor
        item['labels'] = torch.tensor(self.labels[idx]).long()
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IMDbDataset(train_encodings, y_train)
val_dataset = IMDbDataset(val_encodings, y_valid)
test_dataset = IMDbDataset(test_encodings, y_test)

In [11]:
batch_size = 16
learning_rate = 3e-5
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)
valid_dataloader = torch.utils.data.DataLoader(
    val_dataset, shuffle=True, batch_size=batch_size
)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_func = torch.nn.CrossEntropyLoss()
model = model.to(device)
loss_func = loss_func.to(device)



In [66]:
def train_model(model, train_dataloader, optimizer, loss_func):
    model.train()
    train_loss = 0
    for batch in tqdm.tqdm(train_dataloader):
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        loss = loss_func(outputs.logits, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    return train_loss / len(train_dataloader)


In [67]:
def evaluate_model(model, val_dataloader, loss_func):
    model.eval()
    valid_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in tqdm.tqdm(val_dataloader):

            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
           

            outputs = model(input_ids, attention_mask=attention_mask)
            loss = loss_func(outputs.logits, labels)
            valid_loss += loss.item()
            pred = outputs.logits.argmax(dim=1)
            correct += (pred == labels).sum().item()
            
    return  correct / len(val_dataloader.dataset), valid_loss / len(val_dataloader),

In [68]:
epochs = 3
train_losses = []
valid_losses = []
accuracy_train = []
perplexities = []
accuracy, valid_loss = evaluate_model(model, valid_dataloader, loss_func)
print(f"Pré treino; Validation Loss: {valid_loss:.3f}; Perplexity: {np.exp(valid_loss):.3f}; Accuracy: {accuracy:.3f}")
for t in range(epochs):
  train_loss = train_model(model, train_dataloader, loss_func, optimizer)
  train_losses.append(train_loss)
  accuracy, valid_loss = evaluate_model(model, valid_dataloader, loss_func)
  valid_losses.append(valid_loss)
  accuracy_train.append(accuracy)
  perplexities.append(np.exp(train_loss))
  print(f"Epoch: {t+1}; Train Loss: {train_loss:.3f}; Perplexity: {np.exp(train_loss):.3f}; Validation Loss: {valid_loss:.3f}; Accuracy: {accuracy:.3f}")

100%|██████████| 313/313 [02:59<00:00,  1.74it/s]


Pré treino; Validation Loss: 0.702; Perplexity: 2.017; Accuracy: 0.501


  0%|          | 0/1250 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 24.00 MiB (GPU 0; 5.81 GiB total capacity; 4.51 GiB already allocated; 55.38 MiB free; 4.56 GiB reserved in total by PyTorch)

In [12]:
!pip install pytorch-lightning

     |████████████████████████████████| 708 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 419 kB 11.2 MB/s eta 0:00:01
  Using cached torch-1.12.1-cp38-cp38-manylinux1_x86_64.whl (776.3 MB)
     |████████████████████████████████| 5.9 MB 57 kB/s s eta 0:00:01
     |████████████████████████████████| 93 kB 187 kB/s  eta 0:00:01
     |████████████████████████████████| 232 kB 14.2 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 6.6 kB/s  eta 0:00:01
     |████████████████████████████████| 4.9 MB 13.0 MB/s eta 0:00:01
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
ERROR: torchvision 0.6.1+cu101 has requirement torch==1.5.1, but you'll have torch 1.12.1 which is incompatible.
ERROR: google-api-core 2.10.0 has requirement protobuf<5.0.0dev,>=3.20.1, but you'll have protobuf 3.19.6 which is incompatible.
  Attempting uninstall: torch
    Found existing installation: torch 

In [ ]:
import pytorch_lightning as pl

In [ ]:
class BERTpl(pl.LightningModule):
    def __init__(self, num_epochs=3):
        super().__init__()
        self.model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
        self.num_epochs = num_epochs
        self.loss_func = torch.nn.CrossEntropyLoss()
        self.classifier = torch.nn.Sequential(
            torch.nn.Linear(self.model.config.hidden_size, 256),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.Linear(256, 2),
        )

    def forward(self, input_ids, attention_mask, labels):
        outputs = self.model(input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.pooler_output)
        loss = self.loss_func(logits, labels)
        return loss, logits

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        loss, logits = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        loss, logits = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=3e-5)
        return optimizer
    
